<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.css" integrity="sha512-5A8nwdMOWrSz20fDsjczgUidUBR8liPYU+WymTZP1lmY9G6Oc7HlZv156XqnsgNUzTyMefFTcsFH/tnJE/+xBg==" crossorigin="anonymous" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js"></script>

# Creating animations

While the ```microplot``` module allows one to create single-frame multi-channel images with annotations, the ```microanim``` module allows one to create interactive plots and save animations of time-lapse versions of such plots. Both modules use an almost identical set of options to render the images and add annotations. 

In [19]:
#%load_ext autoreload
#%autoreload 2

## Dataset

To create an animation, you need a dataset containing multiple frames. As dataset, you can either use a Numpy array with shape CTXY or use one of the ```dataset``` classes. We start with the simplest case of an array and load a complete stack that we reshape:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io

In [2]:
image = skimage.io.imread('../demodata/coli_nucl_ori_ter.tif')

## Create the animation object

Now we can use default settings to create an animation object. For that we only need the dataset, here in the form of a Numpy array. **If you use such a Numpy array, by default all channels are plotted. If you don't want that either create a dataset object (see later) or slice your array**.

We will have access to some interactivity to scroll through the frames. So before we start we need to activate Matplotlib's interactive widget mode:

In [3]:
%matplotlib widget

In [4]:
from microfilm.microanim import Microanim

In [5]:
image.shape

(3, 30, 220, 169)

Now we can create the animation object. The animation object takes the same arguments as the ```Microimage``` object. The main difference is that you need to provide in addition a ```data``` parameter which represents your time-lapse images. The ```data``` can be either one of the ```microfilm.dataset``` objects or a simple Numpy array with dimensions ```C x Z x X x Y``` like ```image``` used here:

In [6]:
anim = Microanim(data=image, cmaps=['pure_blue','pure_red', 'pure_green'])

## Interactive object

To display our time-lapse image and browse through it, we the simply call the ```ui``` of the object:

In [9]:
anim.output

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "Canvas(toolbar=Toolbar(toolitems=[('Ho…

In [8]:
anim.time_slider

IntSlider(value=0, description='Time', max=29)

## Customizing the plot

As mentioned before, you have access to almost all options that you can use in ```microshow``` for the animation. Here's an example where we pick a specific color map per channel, add a scale bar and adjust the plot size:

In [9]:
anim = Microanim(data=image, cmaps=['pure_blue','pure_red', 'pure_green'],
                unit='um', scalebar_size_in_units=4, scalebar_unit_per_pix=0.065, fig_scaling=2)
anim.ui

### Access to microim

As the animation object is very similar to the simple plot object, you get also access to the same underlying methods e.g. to add a label:

In [10]:
anim.add_label('a comment', label_location='upper right');
anim.ui

And again, you also get access to the underlying Matplotlib objects like axis:

In [11]:
anim.ax.set_aspect(0.7)
anim.ui

## Adding a time-stamp

As a specific feature for animations, you can add a specific label which is a time stamp. At the moment you are limited to use a ```HH:MM:SS``` format. To add it to your plot, you just specify the unit (e.g. ```T``` for minutes, abbreviations are from Pandas), and the number of units per time frame e.g. here 5min:

In [12]:
anim = Microanim(data=image, cmaps=['pure_blue','pure_red', 'pure_green'],
                unit='um', scalebar_size_in_units=4, scalebar_unit_per_pix=0.065, fig_scaling=3)
anim.add_time_stamp('T', 10, location='lower left')
anim.ui

Just like for regular plots, you may want to create animations with multiple panels. This is done in the same way as for figures, by assembling pre-made single animations. Find more information in [this section](create_panels.ipynb).

## Exporting the animation

Once you are satisfied with your movie, you may want to export it to integrate it e.g. in a presentation. You can do that easily using the ```save_movie``` method. You can specify the image quality and the frames per second for various movie formats (.mov, .avi, .mpg, .mpeg, .mp4, .mkv, .wmv) and the image format is simply recovered from the file name. You can also export as gif, but don't have access to the ```quality``` option there:

In [13]:
anim.save_movie('movie.mp4', fps=15, quality=5)

[WARNING: _io: 411 2021-06-27 15:54:26,865] IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (230, 300) to (240, 304) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [14]:
anim.save_movie('movie.gif', fps=15)

## Using the dataset class

Instead of a plain Numpy array you can pass one of the ```dataset``` objects to the animation constructor. You have here the additional flexibility of choosing which channels to plot for example. Also, if you deal with large images, it is favorable to use a format such as H5 where not the full dataset is loaded into memory.

We keep our simple example, and transform our Numpy array into a ```dataset```. We can specify a name for each channel. Here again, the array format **has to be CTXY**.

In [15]:
from microfilm.dataset import Nparray

data = Nparray(nparray=image, channel_name=['nucleoid', 'ori', 'ter'])

Now we can explicitly specify which channels to plot. For example if you just want a single channel:

In [16]:
anim = Microanim(data=data, channels=['nucleoid','ori'], cmaps=['pure_red','pure_green'],
                unit='um', scalebar_size_in_units=4, scalebar_unit_per_pix=0.065, fig_scaling=5)
anim.add_label('A')
anim.add_time_stamp('T', 10, location='lower left')

In [17]:
anim.ui